In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

In [2]:
!conda list scipy

# packages in environment at C:\ProgramData\Anaconda3\envs\DataScienceRetreat:
#
# Name                    Version                   Build  Channel
scipy                     1.1.0            py36h4f6bf74_1  


In [3]:
import os
print("Orig Dir:",os.getcwd())
os.chdir('../../')
print("Current Dir:",os.getcwd())

Orig Dir: C:\Users\p.schambach\Desktop\DSR\drone_steering\models\playground
Current Dir: C:\Users\p.schambach\Desktop\DSR\drone_steering


In [4]:
data_path = 'data/gesture/features/features_flip_c_03_120.csv'
df_orig = pd.read_csv(data_path,sep=',')

In [5]:
df_orig.head()

,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,ms_since_last_frame,ms_since_start,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y
0,341.136444,224.253407,320.787999,274.947119,329.514176,166.895228,345.275067,277.081232,0,0.0,256.999411,225.093688,281.429514,272.660262,262.775556,169.641120,253.241117,271.500238
1,340.985901,224.344366,320.455079,275.406629,329.389916,166.027360,345.465304,277.669048,121,121.0,256.989746,225.065718,281.315210,273.121688,262.903829,169.255270,252.963656,271.032473
2,339.929797,222.910622,320.485332,274.863216,328.574083,164.447614,345.679107,277.607702,124,245.0,257.143566,224.858589,281.088004,273.002894,263.136312,168.805475,253.317534,272.038940
3,341.394664,225.080272,323.362793,277.252602,327.161964,165.040823,345.486792,282.963012,123,368.0,256.015686,224.441979,280.673399,273.297257,262.428639,167.931032,251.685311,271.837686
4,341.007135,227.292845,320.850446,280.183320,326.259811,167.356121,342.353060,285.637142,119,487.0,255.963889,224.110870,280.559333,274.064114,261.804933,166.962579,253.027392,270.509372


In [6]:
from app_local.module import DataFrameInterpolator

In [7]:
di = DataFrameInterpolator()
di.display_information()

In [8]:
new_df = di.scaleDataFrame(df_orig, 64048, 'avg')
new_df = di.get_new_df(new_df, 50)
di.display_information()

FrameLength in Milliseconds: 50 

Original Time:
	Number of Frames: 515
	 [124. 245. 370. 493. 613.] ... [63542. 63671. 63795. 63922. 64048.] 

New Time:
	Number of Frames: 1280
	 [124 174 224 274 324] ... [63874 63924 63974 64024 64074] 

Original DataFrame Shape: (515, 18)
New DataFrame Shape: (1280, 17)


In [11]:
from app_local.module import DataEnsembler
de = DataEnsembler()

de.investigate_available_datafiles(data_dir='data/gesture/')
de.data_source_df

,filename,filetype,movement,person,filenum,frame_length
0,features_flip_c_01_120.csv,features,flip,c,01,120
1,features_flip_c_02_120.csv,features,flip,c,02,120
2,features_flip_c_03_120.csv,features,flip,c,03,120
3,features_flip_l_01_120.csv,features,flip,l,01,120
4,features_flip_l_02_120.csv,features,flip,l,02,120
5,features_flip_l_03_120.csv,features,flip,l,03,120
6,features_flip_p_01_120.csv,features,flip,p,01,120
7,features_land_c_01_120.csv,features,land,c,01,120
8,features_land_c_02_120.csv,features,land,c,02,120
9,features_land_c_03_120.csv,features,land,c,03,120


In [12]:
de.combined_data_files_df

,filename_features,filename_labels
0,features_flip_c_01_120.csv,labels_flip_c_01.csv
1,features_flip_c_02_120.csv,labels_flip_c_02.csv
2,features_flip_c_03_120.csv,labels_flip_c_03.csv
3,features_flip_l_01_120.csv,labels_flip_l_01.csv
4,features_flip_l_02_120.csv,labels_flip_l_02.csv
5,features_flip_l_03_120.csv,labels_flip_l_03.csv
6,features_flip_p_01_120.csv,labels_flip_p_01.csv
7,features_land_c_01_120.csv,labels_land_c_01.csv
8,features_land_c_02_120.csv,labels_land_c_02.csv
9,features_land_c_03_120.csv,labels_land_c_03.csv


In [ ]:
video_stats = pd.read_csv('docs/video_stats.csv', sep="\t")
video_stats